In [286]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator

In [287]:
def print_digit(digit):
    pyplot.imshow(digit.reshape(28,28))

In [288]:
def gini_index(probability):
    return 1.0 - probability**2 - (1.0 - probability)**2

In [289]:
data = pd.read_csv('learn.csv')

data = data[ (data.answer == 9) | (data.answer == 7)]

data = data.sample(10000, axis=0)
data = data.reset_index()
data = data.drop('index', axis=1)

answers = data['answer']
data = data.drop('answer', axis=1)

In [290]:
x_train, x_test, y_train, y_test = train_test_split(data, answers, test_size=0.2, random_state=42)
x_train = np.array(x_train)
x_test = np.array(x_test)

In [291]:
trees_numb = 11
list_size = 4
tree_hight = 40

In [292]:
skl_clf = RandomForestClassifier(n_estimators=trees_numb, max_depth=tree_hight, min_samples_leaf=list_size)
skl_clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [293]:
class tree_node:
    objects = np.array([])
    min_size = 0
    sep_index = 0
    sep_value = 0
    
    def __init__ (self, size=50):
        self.min_size = size
        
    def set_children(self, left, right):
        self.left = left
        self.right = right

In [294]:
class decisive_tree:
    hight = 0
    
    def __init__(self, h):
        self.hight = h
    
    def set_root(self, root):
        self.root = root
    
    def insert(self, node, h, func):
        if h <= self.hight:
            
            separator = self.separator(node, func)
            node.sep_index = separator[1]
            node.sep_value = separator[0]
            
            left_node = tree_node(node.min_size)
            left_node.objects = np.array([obj for obj in node.objects if obj[node.sep_index] <= node.sep_value])
            
            right_node = tree_node(node.min_size)
            right_node.objects = np.array([obj for obj in node.objects if obj[node.sep_index] > node.sep_value])

            node.set_children(left_node, right_node)
            
            if len(left_node.objects) > node.min_size:
                self.insert(left_node, h+1, func)
    
            if len(right_node.objects) > node.min_size:
                self.insert(right_node, h+1, func)
                
    def separator(self, node, func):
        indexes = np.arange(0, func(len(x_train[0])), dtype=int)
        for i in range(0, len(indexes)):
            index = np.random.randint(0, 783)
            elem = np.random.randint(0, len(node.objects))
            while node.objects[elem][index] == 0:
                index = np.random.randint(50, 730)
                elem = np.random.randint(0, len(node.objects))
            indexes[i] = index    
        
        means = np.array([node.objects[:, i].sum() / float(len(node.objects)) - 50 for i in indexes])
        
        entropy = []
        
        for i in range(0, len(indexes)):
            left = []
            right = []
            for obj in node.objects:
                if obj[indexes[i]] <= means[i]:
                    left.append(0.0 if obj[784] == 7 else 1.0)
                else:
                    right.append(0.0 if obj[784] == 7 else 1.0)
                    
            if len(left) != 0 & len(right) != 0:
                entropy.append( gini_index(sum(left) / len(left)) * gini_index(sum(right) / len(right)) )
            elif len(left) != 0:
                entropy.append(gini_index(sum(left) / len(left)) * 1.0)
            else:
                entropy.append(gini_index(sum(right) / len(right)) * 1.0)
          
        index = entropy.index(min(entropy))
        
        return (means[index], indexes[index])

In [295]:
class RandomForestClassifier (BaseEstimator):
    numb_of_trees = 0
    min_list_size = 0
    max_hight = 0
    trees = []
    more = 0
    
    def __init__(self, numb_of_trees=10, max_list_size=3, max_hight=15, q_func=math.sqrt):
        self.numb_of_trees = numb_of_trees
        self.min_list_size = max_list_size
        self.max_hight = max_hight
        self.q_func = q_func
    
    def s_or_n(self, sample):
        summ = sum([1.0 for i in xrange(0, len(sample)) if sample[i, 784] == 7])
        if summ >= len(sample) / 2:
            more = 7
        else:
            more = 9
        
    def fit(self, train, labels):
        train = np.column_stack((train, labels))
        
        while len(self.trees) != self.numb_of_trees:
            sample = np.array([train[np.random.randint(0, len(train)-1)] for i in xrange(0, len(train)/self.numb_of_trees)])
            self.s_or_n(sample)
            
            tree =  decisive_tree(self.max_hight)
            root = tree_node(self.min_list_size)
            root.objects = sample
            
            tree.insert(root, 0, self.q_func)
            tree.set_root(root)
            
            self.trees.append(tree)
            
    def predict(self, x_test):
        answers = []
        
        for obj in x_test:
            verdicts = []
            for tree in self.trees:
                verdicts.append(self.version(tree.root, obj, 0) == 9)
            if sum(verdicts) > len(verdicts) / 2:
                answers.append(9)
            else:
                answers.append(7)
            
        
        return answers
    
    def version(self, node, obj, h):
        if obj[node.sep_index] <= node.sep_value:
            if (len(node.objects) > self.min_list_size) & (h <= self.max_hight):
                return self.version(node.left, obj, h+1)
            else:
                result = sum([1.0 for i in xrange(0, len(node.objects)) if node.objects[i][784] == 7])
                if result > len(node.objects) / 2:
                    return 7
                elif result < len(node.objects / 2):
                    return 9
                else:
                    return self.more
        else:
            if len(node.objects) > self.min_list_size & (h <= self.max_hight):
                return self.version(node.right, obj, h+1)
            else:
                result = sum([1.0 for i in xrange(0, len(node.objects)) if node.objects[i][784] == 7])
                if result > len(node.objects) / 2:
                    return 7
                elif result < len(node.objects / 2):
                    return 9
                else:
                    return self.more
                
    def score(self, x_test, y_test):
        answers = np.array(self.predict(x_test))
        result = sum(y_test[i] == answers[i] for i in range(0, len(y_test))) / float(len(y_test))
        return result

In [296]:
y_test = np.array(y_test)   

rfc = RandomForestClassifier(numb_of_trees=trees_numb, max_list_size=list_size, max_hight=tree_hight)
rfc.fit(x_train, y_train)
accuracy = rfc.score(x_test, y_test)
    
print "My result = " + str(round(accuracy, 4))
print "SKLEARN result = " + str(round(skl_clf.score(x_test, y_test), 4))

My result = 0.9375
SKLEARN result = 0.98
